In [1]:
print("Starting imports...")
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda, float64, complex128
from numba.cuda import jit as cuda_jit
import math

print("Importing few...")
import few

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import KerrEccEqFlux
from few.amplitude.ampinterp2d import AmpInterpKerrEccEq
from few.summation.interpolatedmodesum import InterpolatedModeSum 


from few.utils.ylm import GetYlms

from few import get_file_manager

from few.waveform import FastKerrEccentricEquatorialFlux

from few.utils.geodesic import get_fundamental_frequencies

from few.utils.constants import YRSID_SI

import os
import sys

# Change to the desired directory
os.chdir('/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

# Add it to Python path
sys.path.insert(0, '/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

print("Importing GWfuncs...")
import GWfuncs
# import gc
# import pickle
print("Importing cupy...")
import cupy as cp

print("Configuring few...")
# tune few configuration
cfg_set = few.get_config_setter(reset=True)
cfg_set.set_log_level("info")

print("Importing dynesty...")
import dynesty

Starting imports...
Importing few...
Importing GWfuncs...
Importing cupy...
Configuring few...
Importing dynesty...


In [2]:
for backend in ["cpu", "cuda11x", "cuda12x", "cuda", "gpu"]: 
    print(f" - Backend '{backend}': {"available" if few.has_backend(backend) else "unavailable"}")  

 - Backend 'cpu': available
 - Backend 'cuda11x': unavailable
 - Backend 'cuda12x': available
 - Backend 'cuda': available
 - Backend 'gpu': available


In [3]:
# GPU configuration and missing variables
use_gpu = True
dt = 10     # Time step
T = 1.0     # Total time

# Check GPU availability
if not cuda.is_available():
    print("Warning: CUDA not available, falling back to CPU")
    use_gpu = False

In [4]:
print("Setting up waveform generator...")
# keyword arguments for inspiral generator 
inspiral_kwargs={
        "func": 'KerrEccEqFlux',
        "DENSE_STEPPING": 0, #change to 1/True for uniform sampling
        "include_minus_m": False, 
        "use_gpu" : use_gpu,
        "force_backend": "cuda12x"  # Force GPU
}

# keyword arguments for inspiral generator 
amplitude_kwargs = {
    "force_backend": "cuda12x" # Force GPU
    # "use_gpu" : use_gpu
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "force_backend": "cuda12x",  # Force GPU
    # "assume_positive_m": True  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "force_backend": "cuda12x",  # Force GPU
    "pad_output": True,
    # "use_gpu" : use_gpu
}

print("Creating FastKerrEccentricEquatorialFlux...")
# Kerr eccentric flux
waveform_gen = FastKerrEccentricEquatorialFlux(
    inspiral_kwargs=inspiral_kwargs,
    amplitude_kwargs=amplitude_kwargs,
    Ylm_kwargs=Ylm_kwargs,
    sum_kwargs=sum_kwargs,
    use_gpu=use_gpu,
)

Setting up waveform generator...
Creating FastKerrEccentricEquatorialFlux...


In [5]:
# print("Initializing trajectory and amplitude generators...")
# # Initialize trajectory and amplitude generators
# traj = EMRIInspiral(func=KerrEccEqFlux, force_backend="cuda12x", use_gpu=use_gpu)
# amp = AmpInterpKerrEccEq(force_backend="cuda12x")
# interpolate_mode_sum = InterpolatedModeSum(force_backend="cuda12x", pad_output= True)
# ylm_gen = GetYlms(include_minus_m=False, force_backend="cuda12x")


In [6]:
print("Initializing GravWaveAnalysis class")
gwf = GWfuncs.GravWaveAnalysis(T, dt)

Initializing GravWaveAnalysis class


In [7]:
#Generating data (true)

m1_o = 1e6
m2_o = 1e1
a_o = 0.3
p0_o = 12
e0_o = 0.1
xI_o = 1.0
theta_o = np.pi/3  # polar viewing angle
phi_o = np.pi/4  # azimuthal viewing angle
dist = 1 # Gpc

params_star = (m1_o, m2_o, a_o, p0_o, e0_o, xI_o, theta_o, phi_o, dist)
loglike_obj = GWfuncs.LogLike(params_star, waveform_gen, gwf)
#TODO: fix the arguments logic in GWfuncs

In [8]:
loglike_obj.signal

array([ 6.65436270e-23+7.45156464e-23j,  7.57824631e-23+6.69291801e-23j,
        8.35889827e-23+5.86994339e-23j, ...,
       -4.16048237e-23+8.81427320e-23j, -2.71171756e-23+9.05747103e-23j,
       -1.28885656e-23+9.13684625e-23j], shape=(3155815,))

In [9]:
loglike_obj_alt = GWfuncs.LogLike(params_star, waveform_gen, T=T, dt=dt)
loglike_obj_alt.signal

array([ 6.65436270e-23+7.45156464e-23j,  7.57824631e-23+6.69291801e-23j,
        8.35889827e-23+5.86994339e-23j, ...,
       -4.16048237e-23+8.81427320e-23j, -2.71171756e-23+9.05747103e-23j,
       -1.28885656e-23+9.13684625e-23j], shape=(3155815,))

In [10]:
# Generate sample template
m1_t = 1e5
m2_t = 1e1
a_t = 0.4
p0_t = 10
e0_t = 0.2
xI_t = 1.0
theta_t = np.pi/3  # polar viewing angle
phi_t = np.pi/4  # azimuthal viewing angle
dist_t = 1 # Gpc
params_template = (m1_t, m2_t, a_t, p0_t, e0_t, xI_t, theta_t, phi_t, dist_t)
loglike_temp = loglike_obj(params_template)

In [11]:
loglike_temp

1.8793050836641157e-30

In [12]:

# Parameter space search example
def parameter_space_search_example(n_samples=10):
    """Example parameter space search"""
    print(f"Running parameter space search with {n_samples} samples...")
    
    # Parameter ranges (adjust as needed)
    m1_range = (9.99e5, 1.001e6)
    m2_range = (9.99, 10.001)  
    a_range = (0.2997, 0.3003)
    p0_range = (11.988, 12.012)
    e0_range = (9.99e-2, 0.1001)
    xI0_range = (1.0, 1.0) # keeping it equatorial for now
    theta_range = (np.pi / 3 * (0.999), np.pi / 3 * (1.001))
    phi_range = (np.pi / 4 * (0.999), np.pi / 4 * (1.001))
    dist_range = (0.999, 1.001)  
    
    # Generate random parameter samples
    np.random.seed(7)  # For reproducibility
    
    for i in range(n_samples):
        # Sample masses log-uniformly, others  uniformly
        m1 = 10**(np.random.uniform(np.log10(m1_range[0]), np.log10(m1_range[1])))
        m2 = 10**(np.random.uniform(np.log10(m2_range[0]), np.log10(m2_range[1])))
        params = [
            m1,
            m2, 
            np.random.uniform(*a_range),
            np.random.uniform(*p0_range),
            np.random.uniform(*e0_range),
            np.random.uniform(*xI0_range),
            np.random.uniform(*theta_range),
            np.random.uniform(*phi_range),
            np.random.uniform(*dist_range)
        ]
        
        try:
            # Evaluate likelihood
            f_stat = loglike_obj(params)
            print(f"Sample {i+1}/{n_samples}: f_stat = {f_stat}")
                
        except Exception as e:
            print(f"Error in evaluation {i+1}: {e}")
            continue
    
    print("Parameter space search completed!")

In [13]:
print("All initialization complete! Starting parameter_space_search_example...")
parameter_space_search_example(n_samples=100)

All initialization complete! Starting parameter_space_search_example...
Running parameter space search with 100 samples...
Sample 1/100: f_stat = -6.1735183341265504e-27
Sample 2/100: f_stat = -0.002361997755041137
Sample 3/100: f_stat = 1.0373242334730765e-28
Sample 4/100: f_stat = -1.9421081154465682e-14
Sample 5/100: f_stat = 2.4311588394976803e-29
Sample 6/100: f_stat = -inf
Sample 7/100: f_stat = 1.3340025908656556e-27
Sample 8/100: f_stat = 8.903509115387706e-14
Sample 9/100: f_stat = -0.00020442943380584496
Sample 10/100: f_stat = 2.1167256216706214e-14
Sample 11/100: f_stat = -0.00017659843070256485
Sample 12/100: f_stat = 1.1080942149801563e-28
Sample 13/100: f_stat = -1.659383226324194e-27
Sample 14/100: f_stat = -6.888987896261919e-42
Sample 15/100: f_stat = -0.0011763335010707438
Sample 16/100: f_stat = -1.696213413340572e-05
Sample 17/100: f_stat = 0.0036948839551851796
Sample 18/100: f_stat = 8.044696641649256e-28
Sample 19/100: f_stat = 0.0004506364017871352
Sample 20/10

In [14]:
## PRIOR: masses log-uniform
def prior_transform(utheta):
      um1, um2, ua, up0, ue0, uxI0, utheta_angle, uphi, udist = utheta

      # Parameter limits
      m1lim = [9.99e5, 1.001e6]
      m2lim = [9.99, 10.01]
      alim = [0.2997, 0.3003]
      p0lim = [11.988, 12.012]
      e0lim = [9.99e-2, 0.1001]
      xI0lim = [1.0, 1.0]
      thetalim = [np.pi / 3 * (0.999), np.pi / 3 * (1.001)]
      philim = [np.pi / 4 * (0.999), np.pi / 4 * (1.001)]
      distlim = [0.999, 1.001]  # Distance in Gpc

      # Log-uniform for masses
      m1 = 10**(np.log10(m1lim[0]) + um1 * (np.log10(m1lim[1]) - np.log10(m1lim[0])))
      m2 = 10**(np.log10(m2lim[0]) + um2 * (np.log10(m2lim[1]) - np.log10(m2lim[0])))

      # Uniform for other parameters
      a = (alim[1] - alim[0]) * ua + alim[0]
      p0 = (p0lim[1] - p0lim[0]) * up0 + p0lim[0]
      e0 = (e0lim[1] - e0lim[0]) * ue0 + e0lim[0]
      xI0 = 1.0  # Fixed value
      theta = (thetalim[1] - thetalim[0]) * utheta_angle + thetalim[0]
      phi = (philim[1] - philim[0]) * uphi + philim[0]
      dist = (distlim[1] - distlim[0]) * udist + distlim[0]

      return m1, m2, a, p0, e0, xI0, theta, phi, dist

In [15]:
rstate = np.random.default_rng(7)
with dynesty.pool.Pool(16, loglike_obj, prior_transform) as pool:
    dsampler = dynesty.NestedSampler(
        loglike_obj,  
        prior_transform,
        ndim=9,
        bound='multi',
        sample='rwalk',
        rstate=rstate
    )
    dsampler.run_nested(checkpoint_file='dynestystatic.save')

64it [01:16,  1.20s/it, bound: 0 | nc: 1 | ncall: 569 | eff(%): 11.248 | loglstar:   -inf < -0.000 <    inf | logz: -2.372 +/-    nan | dlogz:  2.408 >  0.509]

KeyboardInterrupt

